# Function

In [ ]:
from weakref import ref
import numpy as np
import torch

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.normal import Normal
from torchvision import transforms
from torch.nn.modules.distance import PairwiseDistance
from copy import copy
import matplotlib.pyplot as plt

from datetime import time, date, datetime
import time
import pickle
import random
import sys
import itertools

import math
import matplotlib
from matplotlib.colors import ListedColormap

from mpl_toolkits.axisartist.axislines import SubplotZero
import warnings
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

import scipy
import scipy.io as io 
import copy

from mpl_toolkits.axisartist.axislines import SubplotZero
from scipy.integrate import quad
from scipy.stats import norm
from scipy.interpolate import interp1d
from sympy.combinatorics.graycode import GrayCode
import pickle as pkl
import os

from IPython import display
from pathlib import Path
from IPython.core.pylabtools import figsize
from numpy.core.fromnumeric import size



class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

def binary_batch_to_decimal(Targets):
    """
    This Function Takes a Matrix of the size N * S
    - N number of symbols
    - S Symbol size (#Bits of each symbol )
    and returns a tensor of integers 
    Example: 
    Targets: [[0, 1, 1, 0, 0, 1],
              [1, 0, 1, 1, 0, 1],
              [1, 1, 1, 0, 1, 1]]
    int_Targets: [3, 5, 2, 1, 7]
    """
    int_Targets = np.zeros([len(Targets[:,0])])    
    for j in range(len(Targets[:,0])):
        x=0
        l=len(Targets[0,:])
        for i in range(l):
                x+=Targets[j,i]*2**(l-1-i) 
        int_Targets[j]=x
    return int_Targets.reshape(-1,1)

def tx_PAM_Gen(ModuFormat, b_size): 
  '''
  This fuction generates PAM symbols 
  input: 
    sys: attribute dictionary of system parameters 
  output:
    ref_Bit: the random bits generated
    ref_PAM: the PAM optical coordiantes (any of [0,1,2,3])
  '''
  
  # ModuFormat = syst.PMA_order

  ## bit generation

  rbs = np.random.randint(0,2,(b_size,1))

  k_bits = np.log2(ModuFormat)
  Nsymbs = np.floor(np.max(np.size((rbs)))/k_bits)

  ref_BIT = rbs[0:int(Nsymbs*k_bits)]
  
  ## PAM level generation
  bit_matrix = ref_BIT.reshape(int(Nsymbs),int(k_bits))
  tx_vec = binary_batch_to_decimal(bit_matrix)
  

  ref_PAM = tx_vec # pammod(tx_vec,ModuFormat);

  return ref_BIT,ref_PAM

def rectpulse(ref_PAM, syst):
	"""
		Parameters
		------------
		ref_PAM: a sequence of PAM symbols for examplpe for PAM 4 [0,1,2,3] 
    syst: atribute dictionary of the system 
      syst.puls_sps is used here which determines the sample per symbol ratio
		samplesPerSymbol

		Returns
		------------
		outNRZ: NRZ train pulses. 
	"""
	outNRZ = [ref_PAM[i] for i in range(0,len(ref_PAM)) for j in range(0,syst.puls_sps)]
	outNRZ = np.array(outNRZ) 	
	return outNRZ

def add_laser_PN(Data_in_Func, laser): 
  NoiseSamp = np.random.randn(np.max(np.size(Data_in_Func)), 2)
  PNt = (2*np.pi*laser.tx_linewidth*laser.d_t)*np.cumsum(NoiseSamp[:,0]).reshape(-1,1) # considers only firtst dimention of the NoiseSamp
  Data_out_Laser = np.sqrt(laser.power)*Data_in_Func*np.exp(1j*PNt) # transmitter phase noise

  pow_total = np.mean(np.abs(Data_out_Laser)**2);
  # print('******Laser*********');  
  # print('Laser Linewidth = ',(laser.tx_linewidth/1e6),' MHz\n')  
  
  return Data_out_Laser

def add_fiber_CD(Data_in_Func, fiber): 
  w_T = 2*np.pi*np.array(list(np.arange(0,np.max(np.size(Data_in_Func)/2)))+list(np.arange(-np.max(np.size(Data_in_Func))//2,0))).reshape(-1,1)/(np.max(np.size(Data_in_Func))/fiber.BW)
  w = w_T # +min(w_T);  
  HDispersion=np.exp(1j*(fiber.beta2*w**2*fiber.Length)/2+1j*(fiber.beta3*w**3*fiber.Length)/6)  
  # HDispersion=exp(1i*(fiber.beta2*w.^2*fiber.Length)/2);
  # HDispersion2=exp(-1i*(fiber.beta2*w.^2*fiber.Length)/2);
  # HDispersion=HDispersion1.*HDispersion2;
  f_fiber = np.fft.fftshift(w_T/(2e9*np.pi))
  # plt.plot(f_fiber, np.fft.fftshift((fiber.beta2*w**2*fiber.Length)/2+1j*(fier.beta3*w**3*fiber.Length)/6));
  
  # plt.xlabel('frequency (GHz)')
  # plt.ylabel('Fiber Phase Response (fiber.Dispersion caused)')
  # plt.title('Dispersion = {} ps/(nm*km)'.format(fiber.Dispersion))
  # plt.grid(True)
  # plt.show()

  Data_out_Fiber = np.fft.ifft(np.fft.fft(Data_in_Func.reshape(1,-1))*HDispersion.reshape(1,-1))
  alpha=10**(0.05*fiber.Attenuation*fiber.Length);
  # %         Data_out_Fiber=Data_out_Fiber/sqrt(mean(abs(Data_out_Fiber.^2)));
  Data_out_Fiber=Data_out_Fiber.reshape(-1,1)/alpha;

  # print('******Fiber*********');
  # print('Wavelength = {} nm'.format(fiber.lambdaa))
  # print('Fiber Length = {} Km'.format(fiber.Length))
  # print('Dispersion = {} ps/(nm*km)'.format(fiber.Dispersion))
  # print('Attenuation = {} dB/km'.format(fiber.Attenuation))

  pow_total=np.mean(np.abs(Data_out_Fiber)**2);
  # print('***** Totoal Power Fiber Out *****\n      ', pow_total,'\n')
  return Data_out_Fiber

def add_ASE(Data_out, syst):
  noise = ((1/np.sqrt(2)) * (np.random.randn(np.max(Data_out.shape)) + 1j * np.random.randn(np.max(Data_out.shape)))).reshape(-1,1)
  awgn = (np.sqrt(np.mean(np.abs(Data_out)**2)*(10**(-syst.ASE_SNR/10))) * noise)
  Data_out_ASE = Data_out + awgn
  return Data_out_ASE

def PAM_resample_demodulate(Data_out, syst): 
    
  # IMDD constellation generation for to be used for detection 
  PAM_const = (np.sqrt(np.array([0,1,2,3]))).reshape(-1,1)
  PAM_const_norm = PAM_const

  # resampling
  PAM_rx=Data_out[int(syst.puls_sps/2):-1:syst.puls_sps].reshape(1,-1)#sampling at half symbol duration

  # demodulate
  RX = []
  for i in range(np.max(PAM_rx.shape)):
    idx = np.argmin(np.abs(PAM_rx[0,i]-PAM_const_norm**2))       
    RX.append(idx)  
  RX = np.array(RX).reshape(-1,1)
  return RX

def messages_to_onehot(intMessages,M):
    """
    Convert messages represented as indexes to one-hot encoding.

    :param messages: List of messages to convert.
    :param order: Number of possible messages.
    :return: One-hot encoded messages.

    Example: messages_to_onehot(torch.tensor([0, 2, 0, 3, 4])) =>>
    torch.tensor([
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],

        [0., 0., 0., 0., 1.],
    ])
    """
    return torch.nn.functional.one_hot(intMessages, num_classes=M).float()

def get_sequential_data(x,seq_len, M):    
  
  win = []
  for i in range(torch.max(torch.tensor(x.shape))-seq_len+1):
    win.append(x[0,i:i+seq_len])
  win = torch.stack(win).reshape(1,-1)
  # print(win.shape)
  onehotTx = messages_to_onehot(win,M)
  win_onehotTx = onehotTx.reshape(-1,M*seq_len)
  return win_onehotTx

def get_SER_BER_MC(model, trP):
    """
    This function receives a constellation and a calculates
    Monte-Carlo BER 
    """
    
    
    ######################################################################
    # Initialization
    ######################################################################    
    bit_errors = 0
    bit_stream_size = 0
    N_symbols = 0
    sym_error_count = 0
    clip = torch.tensor((trP.seq_len-1)*2)

    while sym_error_count < 100:
      ###      
      ######################################################################
      # Create Random data and convert it to one-hot
      ######################################################################
      ref_BIT,ref_PAM = tx_PAM_Gen(trP.M, trP.TSTbatchSize)  
      ref_PAM = torch.tensor(ref_PAM).reshape(1,-1).long().to("cuda")            
      oneHot_seq = get_sequential_data(ref_PAM,trP.seq_len, trP.M).to("cuda")    

      # print('ref pam',ref_PAM.shape)
      # print('\nTxone hot: ',oneHot_seq)

      Tx = ref_PAM[0,:torch.max(torch.tensor(ref_PAM.shape))-clip]
      # print('\ntx: ',Tx[:10])
      N_symbols += torch.max(torch.tensor(Tx.shape))
      # print('n s: ', N_symbols)
      ######################################################################
      # Apply random data to the model
      ######################################################################
      model.eval()
      with torch.no_grad():        
        scores, encOutNorm, channelOut  = model(oneHot_seq)

        # print('\nscores: ',scores.shape)

        m = nn.Softmax(dim=1)
        probabilities = m(scores)
        # print('\nprobabilities: ',probabilities.shape)
      

        intRx = []
        # predictions = torch.zeros([len(probabilities), len(probabilities[0])], dtype=torch.float)       
        
        for i in range (len(probabilities)):  
          maximum_val, idx = torch.max(probabilities[i],dim=0)
          # predictions[i][idx]=1
          intRx.append(idx)
            

  
        # intRx = torch.tensor([torch.where(r==1)[0][0] for r in predictions])
        
        intRx = torch.stack(intRx)        
        # print('\nRX: ',intRx[:10])
        # for i in range(len(intRx)): 
        #     if Tx[i] != intRx[i]:
        #         sym_error_count += 1
        # print(sym_error_count)
        # sys.exit()
        
        sym_error_count+=torch.sum(Tx != intRx)
        # print('EEERRROOOORSSSS: ',sym_error_count)
    SER = sym_error_count/N_symbols  
  
    return SER

def set_attributes(): 
  ##############################################################
  # System parameters
  syst = AttrDict()    
  syst.baud_rate  = 10e9
  syst.PMA_order  = 4
  syst.roll_off   = 1
  syst.puls_span  = 128
  syst.puls_sps   = 20
  syst.ASE_SNR    = 10
  ##############################################################
  # DAC parameters
  DAC = AttrDict()    # DAC parameters
  DAC.sampling_rate = syst.baud_rate*syst.puls_sps
  
  ##############################################################
  # fiber parameters
  fiber = AttrDict()   
  fiber.BW  = DAC.sampling_rate
  fiber.Length      = 3    # [KM]
  fiber.Dispersion  = 18   # [ps/(nm*km)]   (zero at 1310nm  and <=18 at 1550nm)
  fiber.S           = 0    # [ps/(nm^2.km)] Dispersion slope
  fiber.Attenuation = 0    # 0.2; % [dB/km]        (0.34 at 1310nm  and 0.2  at 1550nm)
  fiber.lambdaa     = 1550 # [nm] wavelength
  fiber.beta2       =-1*(fiber.Dispersion*1e-12*1e9*fiber.lambdaa**2*1e-18)/(2*np.pi*3e8)
  fiber.beta3       =-1*(fiber.S*1e-12*1e18*fiber.lambdaa**4*1e-36)/(2*np.pi*3e8)**2

  ##############################################################
  # laser parameters
  laser = AttrDict()
  laser.power = torch.tensor(10**(0/10)) # 0 dBm
  laser.tx_linewidth = 1.6e8 #laser line width  
  laser.d_t = 1/(syst.baud_rate)

  ##############################################################
  # training parameters
  trP = AttrDict()
  trP.M = syst.PMA_order

  return syst, DAC, fiber, laser, trP




# Autoencoder model

In [ ]:
############################################################
## Class of Parameters
class ConstellationNet(nn.Module):
    """
    This Class creates an Autoencoder srtucture which is used to 
    train an End-to-End communication system.
    """
    def __init__(self, syst=AttrDict(), DAC=AttrDict(), fiber=AttrDict(), laser=AttrDict(), trP=AttrDict()):
        """
        Create an Autoencoder.

        :param order: Order of the constellation, i.e. the number of 
        messages that are to be transmitted, or equivalently the number
        of symbols whose placements in the constellation have to be 
        learned.

        :param encoder_layers: Shape of the encoder’s hidden layers. 
        The size of this sequence is the number of hidden layers, with
        each element being a number which specifies the number of 
        neurons in its channel.

        :param decoder_layers: Shape of the decoder’s hidden layers. 
        Uses the same convention as `encoder_layers_sizes` above.
        """
        super(ConstellationNet, self).__init__()
        
        self.syst = syst
        self.DAC = DAC
        self.fiber = fiber
        self.laser = laser
        self.trP = trP


        self.encoder_layers = trP.layers
        self.decoder_layers = self.encoder_layers[::-1]
      
        self.prev_layer_size = self.trP.M*self.trP.seq_len
        self.encoder_layers_list = []

        for self.layer_size in self.encoder_layers:
            self.encoder_layers_list.append(nn.Linear(self.prev_layer_size, self.layer_size,bias=True))            
            self.encoder_layers_list.append(nn.BatchNorm1d(self.layer_size))            
            self.encoder_layers_list.append(trP.activation)            
            
            self.prev_layer_size = self.layer_size

        self.encoder_layers_list += [nn.Linear(self.prev_layer_size, 1), ]

        self.encoder = nn.Sequential(*self.encoder_layers_list)
        #########################################################################
        # Build the decoder network taking the noisy I/Q vector received from
        # the channel as input and outputting a probability vector for each
        # original message. The network additionally uses hidden layers as
        # specified in `decoder_layers`
        #########################################################################
        self.prev_layer_size = 1*self.trP.seq_len
        self.decoder_layers_list = []

        for self.layer_size in self.decoder_layers:
            self.decoder_layers_list.append(nn.Linear(self.prev_layer_size, self.layer_size,bias=True))
            self.decoder_layers_list.append(nn.BatchNorm1d(self.layer_size))
            self.decoder_layers_list.append(trP.activation)
                     
            self.prev_layer_size = self.layer_size
           
        self.decoder_layers_list.append(nn.Linear(self.prev_layer_size, self.trP.M))    
  
        self.decoder = nn.Sequential(*self.decoder_layers_list)
        
    def normalize_power_real(self, x):
        """
        Normalization if constellation received in x is acomplished here.
        """ 
        epsilon = 1e-12        
        average_power = (torch.sum(x**2))/(len(x))
        average_power = torch.max(torch.tensor([epsilon, average_power]))            
        normF = torch.rsqrt(average_power)
        x = x * normF       
        return  x
   

    def get_decInp_win(self, x): 
      a = x.reshape(-1)
      
      win=[]
      for i in range(0,((len(a)-self.trP.seq_len)+1),1):
        # print(i)
        win.append(a[i:i+self.trP.seq_len])
        
      # print(win)
      win = torch.stack(win)
      ww = win.reshape(-1,self.trP.seq_len)
      # print(ww)
      del win
      return ww

    def rectpulse(self, x):
      """
        Parameters
        ------------
        ref_PAM: a sequence of PAM symbols for examplpe for PAM 4 [0,1,2,3] 
        syst: atribute dictionary of the system 
          syst.puls_sps is used here which determines the sample per symbol ratio
        samplesPerSymbol

        Returns
        ------------
        outNRZ: NRZ train pulses. 
      """
      outNRZ = [x[i] for i in range(0,len(x)) for j in range(0,syst.puls_sps)]
      outNRZ = torch.tensor(outNRZ) 	
      return outNRZ
        
    def add_laser_PN(self, Data_in_Func): 
      Data_in_Func = Data_in_Func.reshape(-1,1)
      NoiseSamp = torch.randn(torch.max(torch.tensor(Data_in_Func.shape)), 2)
      
      PNt = (2*torch.pi*self.laser.tx_linewidth*self.laser.d_t)*torch.cumsum(NoiseSamp[:,0],dim=0).reshape(-1,1) # considers only firtst dimention of the NoiseSamp

      Data_out_Laser = torch.sqrt(self.laser.power)*Data_in_Func*torch.exp(1j*PNt) # transmitter phase noise
      # pow_total = torch.mean(torch.abs(Data_out_Laser)**2);
      # # print('******Laser*********');  
      # # print('Laser Linewidth = ',(laser.tx_linewidth/1e6),' MHz\n')  
      
      return Data_out_Laser

    def add_fiber_CD(self, Data_in_Func): 
      w_T = 2*torch.pi*torch.tensor(list(torch.arange(0,torch.max(torch.tensor(Data_in_Func.shape)/2)))+list(torch.arange(-torch.max(torch.tensor(Data_in_Func.shape))//2,0))).reshape(-1,1)/(torch.max(torch.tensor(Data_in_Func.shape))/self.fiber.BW)
      w = w_T # +min(w_T);  
      HDispersion=torch.exp(1j*(self.fiber.beta2*w**2*self.fiber.Length)/2+1j*(self.fiber.beta3*w**3*self.fiber.Length)/6)  
      # HDispersion=exp(1i*(fiber.beta2*w.^2*fiber.Length)/2);
      # HDispersion2=exp(-1i*(fiber.beta2*w.^2*fiber.Length)/2);
      # HDispersion=HDispersion1.*HDispersion2;
      f_fiber = torch.fft.fftshift(w_T/(2e9*torch.pi))
      # plt.plot(f_fiber, np.fft.fftshift((fiber.beta2*w**2*fiber.Length)/2+1j*(fier.beta3*w**3*fiber.Length)/6));
      
      # plt.xlabel('frequency (GHz)')
      # plt.ylabel('Fiber Phase Response (fiber.Dispersion caused)')
      # plt.title('Dispersion = {} ps/(nm*km)'.format(fiber.Dispersion))
      # plt.grid(True)
      # plt.show()

      Data_out_Fiber = torch.fft.ifft(torch.fft.fft(Data_in_Func.reshape(1,-1))*HDispersion.reshape(1,-1))
      alpha=10**(0.05*self.fiber.Attenuation*self.fiber.Length);
      # %         Data_out_Fiber=Data_out_Fiber/sqrt(mean(abs(Data_out_Fiber.^2)));
      Data_out_Fiber=Data_out_Fiber.reshape(-1,1)/alpha;

      # print('******Fiber*********');
      # print('Wavelength = {} nm'.format(fiber.lambdaa))
      # print('Fiber Length = {} Km'.format(fiber.Length))
      # print('Dispersion = {} ps/(nm*km)'.format(fiber.Dispersion))
      # print('Attenuation = {} dB/km'.format(fiber.Attenuation))

      pow_total=torch.mean(torch.abs(Data_out_Fiber)**2);
      # print('***** Totoal Power Fiber Out *****\n      ', pow_total,'\n')
      return Data_out_Fiber

    def add_ASE(self, Data_out):
      noise = ((1/torch.sqrt(torch.tensor(2))) * (torch.randn(torch.max(torch.tensor(Data_out.shape))) + 1j * torch.randn(torch.max(torch.tensor(Data_out.shape))))).reshape(-1,1)
      awgn = (torch.sqrt(torch.mean(torch.abs(Data_out)**2)*(10**(-self.syst.ASE_SNR/10))) * noise)
      Data_out_ASE = Data_out + awgn
      return Data_out_ASE

    def PAM_resample(self, Data_out): 
  

      # resampling
      PAM_rx=Data_out[int(self.syst.puls_sps/2):-1:self.syst.puls_sps].reshape(1,-1)#sampling at half symbol duration

  
      return PAM_rx

    def forward(self, x):
        """
        Performs encoding and decoding of an input vector and compute its
        reconstructed vector.

        :param x: Original one-hot encoded data.
        
        :return: Reconstructed vector. This Vector Containd Scores of be ing '1' or '0'
        """        
        # print('x', x.shape)
        symbols = self.encoder(x) 
        # print('symbols',symbols.shape)
        normalizedSymbols = self.normalize_power_real(symbols)
        # print('normalizedSymbols',normalizedSymbols.shape)
        # plt.plot(normalizedSymbols) 
        PAM_shaped = self.rectpulse(normalizedSymbols)
        # print('PAM_shaped', PAM_shaped.shape)
        noisy_x = self.add_laser_PN(PAM_shaped)
        # print('noisy_x Laser', noisy_x.shape)
        noisy_x = self.add_fiber_CD(noisy_x)
        # print('noisy_x CD', noisy_x.shape)
        noisy_x = self.add_ASE(noisy_x)
        # print('noisy_x ASE', noisy_x.shape)
        noisy_x = torch.abs(noisy_x)**2
        # print('noisy_x FD', noisy_x.shape)
        noisy_x = self.PAM_resample(noisy_x).reshape(-1,1)
        # print('noisy_x RESAMP', noisy_x.shape)
        decInp = self.get_decInp_win(noisy_x).to("cuda")
        # print('decInp', noisy_x.shape)

        
        # sys.exit()
        
        dc = self.decoder(decInp)     
        return dc, normalizedSymbols, noisy_x 


#Train

In [ ]:
torch.set_printoptions(threshold=10_000)
np.random.seed(123)
# torch.random.manual_seed(123)
def train(syst, DAC, fiber, laser, trP):#, EncModel, DecModel):
   
  print('\t\t ---- Training Started!!! ----')
  
  ##############################################################
  ## Get an instance of model
  model = ConstellationNet(syst, DAC, fiber, laser, trP).to("cuda")
  for layer in model:
    if isinstance(layer, nn.Linear):
        nn.init.xavier_uniform_(layer.weight)

  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.RMSprop(model.parameters(), lr=trP.learningRate)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
      optimizer, 
      verbose=True, #  If True, prints a message to stdout for each update. Default: False.
      factor=0.25, # Factor by which the learning rate will be reduced. new_lr = lr * factor. Default: 0.1.
      patience=100, # Number of epochs with no improvement after which learning rate will be reduced. 
      cooldown=50, # Number of epochs to wait before resuming normal operation after lr has been reduced. Default: 0.
      threshold=1e-8 # Threshold for measuring the new optimum, to only focus on significant changes. Default: 1e-4.
  )
  ##############################################################
  clip = torch.tensor((trP.seq_len-1)*2)

  
  i = 0
  losses = []
  best_ser = float('inf')
  while i <= trP.itrs:                
    ## PAM level generation (1-sps)
    ref_BIT,ref_PAM = tx_PAM_Gen(trP.M, trP.batchSize)  
    ref_PAM = torch.tensor(ref_PAM).reshape(1,-1).long().to("cuda")
    oneHot_seq = get_sequential_data(ref_PAM,trP.seq_len, trP.M).to("cuda")    

    model.train()   
    model.eval()
    optimizer.zero_grad()    
    predictions, normalizedSymbols, noisy_x = model(oneHot_seq)   
       
    loss = criterion(predictions, ref_PAM[0,:torch.max(torch.tensor(ref_PAM.shape))-clip])  
    losses.append(loss.to("cpu").detach().numpy())

    loss.backward()        
    optimizer.step()
    scheduler.step(loss) ### Does the Update
    
    if i%(100)== 0:
      ser = get_SER_BER_MC(model, trP)     
      if ser < best_ser: 
        best_ser = ser   
        plt.figure(figsize=(4,1),dpi=100)
      
      plt.scatter(noisy_x.detach().numpy(), np.zeros_like(noisy_x.detach().numpy()),s=20, alpha = .5, label='Received intensity');       
      plt.scatter(normalizedSymbols.to("cpu").detach().numpy()**2,np.zeros_like(normalizedSymbols.to("cpu").detach().numpy())**2,s=5, alpha = .5, label ='Transmitted intensity');       
     
       
      print('\n\t SNR: {}'.format(syst.ASE_SNR))  
      
      print('\n\t Itteration number: {}'.format(i))        
      print('\t Current Loss is: {}'.format(loss))        
      # print('\t Current movement is: {}'.format(total_change)) 
      print('\t Current SER is: {}'.format(ser)) 
      print('\t Best SER is: {}'.format(best_ser)) 
      # print('\n\t current constellatiuon is: {}'.format(prev_constel))        

      
      plt.show()
      plt.figure(figsize=(2,2),dpi=100)
      plt.xlabel('epoch')
      plt.ylabel('loss')
      plt.plot(losses)
      plt.show()
          
    i=i+1
  print('\n\n\t\t\tTraining is done')   
  print('\n\n########### Final Best Results #########################')
  print('****** Bests are ******')

  return model, best_ser

def main():
  syst, DAC, fiber, laser, trP = set_attributes()  
  

  
  trP.learningRate = 0.005
  trP.layers = (64, )
  trP.activation  = nn.SELU()  
  trP.leastMovement = 0.0005
  trP.itrs = 1500
  trP.seq_len = 1
  trP.batchSize = 1024 * trP.seq_len # to ensure that stream is devidable into winSize
  trP.TSTbatchSize = trP.M * 2048 * trP.seq_len # to ensure that stream is devidable into winSize

  SERs = []
  for syst.ASE_SNR in range(12,13,1):        
    model = None
    model, ser = train(syst, DAC, fiber, laser, trP)
    SERs.append(ser)
  SERs = torch.stack(SERs)
  print(SERs)

if __name__ == "__main__":    
  main()

		 ---- Training Started!!! ----


TypeError: ignored